In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torch

In [0]:
# GPU Check

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())
print(torch.cuda.get_device_name(device))

Available devices  1
Current cuda device  0
Tesla K80


In [0]:
# TPU Check

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

In [0]:
import os
import time
import torch
import torchtext
from torchtext import data
from torchtext import datasets
from torchtext.vocab import GloVe
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
import pandas as pd

In [0]:
import random

SEED = 1234 # Random Seed for reproductivity

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField()

## AG_news

The AG's news topic classification dataset is constructed by choosing 4 largest classes from the original corpus. Each class contains 30,000 training samples and 1,900 testing samples. The total number of training samples is 120,000 and testing 7,600.

label
1: World
2: Sports
3: Business
4: Sci/Tech

In [0]:
from torchtext.data import TabularDataset

# put the train and text csv to PATH
DATA_PATH = '/content/drive/My Drive/lottery-ticket-hypothesis-for-text-classification/data/ag_news_csv/'

train = data.TabularDataset(path = DATA_PATH + 'train.csv', format = 'csv', fields = [('text', TEXT), ('label', LABEL)], skip_header = False)
test = data.TabularDataset(path = DATA_PATH + 'test.csv', format = 'csv', fields = [('text', TEXT), ('label', LABEL)], skip_header = False)

# make splits for data
# by default this splits 70:30
train, valid = train.split(split_ratio = 0.7, random_state = random.seed(SEED))

In [0]:
print(f'Number of training examples: {len(train)}')
print(f'Number of training examples: {len(valid)}')
print(f'Number of testing examples: {len(test)}')

print(vars(train.examples[0]))
print(vars(valid.examples[0]))
print(vars(test.examples[0]))

Number of training examples: 84000
Number of training examples: 36000
Number of testing examples: 7600
{'text': ['In', '30', 'sorties', 'on', 'Monday', ',', 'U.S.', 'helicopters', 'delivered', 'more', 'than', '60,000', 'pounds', 'of', 'water', ',', 'medical', 'supplies', 'and', 'food', '.'], 'label': '1'}
{'text': ['London', '-', 'US', 'sports', 'tycoon', 'Malcolm', 'Glazer', 'was', 'trying', 'to', 'win', 'over', 'Manchester', 'United', '#', '39;s', 'two', 'key', 'Irish', 'shareholders', 'in', 'his', 'bid', 'to', 'buy', 'the', 'English', 'soccer', 'club', ',', 'but', 'had', 'not', 'yet', 'entered', 'serious', 'talks', 'with', 'them', ',', 'sources', 'familiar', 'with', 'the', 'situation', 'said', 'yesterday', '.'], 'label': '2'}
{'text': ['Unions', 'representing', 'workers', 'at', 'Turner', '  ', 'Newall', 'say', 'they', 'are', "'", 'disappointed', "'", 'after', 'talks', 'with', 'stricken', 'parent', 'firm', 'Federal', 'Mogul', '.'], 'label': '3'}


In [0]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train)

print ("Length of Text Vocabulary: " + str(len(TEXT.vocab)))
print ("Vector size of Text Vocabulary: ", TEXT.vocab.vectors.size())
print ("Label Length: " + str(len(LABEL.vocab)))

Length of Text Vocabulary: 25002
Vector size of Text Vocabulary:  torch.Size([25002, 100])
Label Length: 4


In [0]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f1595bc7158>, {'2': 0, '1': 1, '4': 2, '3': 3})


In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iter, valid_iter, test_iter = data.BucketIterator.splits(
    (train, valid, test), 
    sort_key = lambda x: x.text,
    batch_size = BATCH_SIZE,
    device = device)

vocab_size = len(TEXT.vocab)

In [0]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 4


In [0]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 120892), (',', 100935), ('.', 91944), ('-', 68725), ('to', 66932), ('a', 66171), ('of', 62242), ('in', 53173), ('and', 45921), ('on', 32689), (' ', 30797), ('for', 26032), ('#', 23694), ('that', 18925), ('39;s', 17430), ('The', 16755), ('(', 16543), (')', 16261), ('with', 14892), ('its', 14611)]


In [0]:
print(TEXT.vocab.itos[:10])
print(LABEL.vocab.stoi)

['<unk>', '<pad>', 'the', ',', '.', '-', 'to', 'a', 'of', 'in']
defaultdict(<function _default_unk_index at 0x7f1595bc7158>, {'2': 0, '1': 1, '4': 2, '3': 3})


In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F

# CNN

In [0]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        text = text.permute(1, 0)
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,591,704 trainable parameters


In [0]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.4598,  1.1272, -1.1509,  ..., -0.6062,  0.1021,  0.8086],
        [ 0.1967, -2.3082,  0.2116,  ...,  0.9473,  0.1453, -0.7713],
        [ 0.0786,  0.3827, -1.8876,  ..., -0.1305, -0.2890, -0.5978]])

In [0]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [0]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text)
        
        loss = criterion(predictions, batch.label)
        
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text)
            
            loss = criterion(predictions, batch.label)
            
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [0]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iter, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


Epoch: 01 | Epoch Time: 0m 14s
	Train Loss: 0.127 | Train Acc: 95.59%
	 Val. Loss: 0.330 |  Val. Acc: 90.84%
Epoch: 02 | Epoch Time: 0m 14s
	Train Loss: 0.107 | Train Acc: 96.27%
	 Val. Loss: 0.361 |  Val. Acc: 90.88%
Epoch: 03 | Epoch Time: 0m 14s
	Train Loss: 0.089 | Train Acc: 96.95%
	 Val. Loss: 0.399 |  Val. Acc: 90.24%
Epoch: 04 | Epoch Time: 0m 14s
	Train Loss: 0.078 | Train Acc: 97.37%
	 Val. Loss: 0.426 |  Val. Acc: 90.39%
Epoch: 05 | Epoch Time: 0m 14s
	Train Loss: 0.069 | Train Acc: 97.65%
	 Val. Loss: 0.463 |  Val. Acc: 90.38%


In [0]:
# EPOCH = 5

model.load_state_dict(torch.load('tut5-model.pt'))

test_loss, test_acc = evaluate(model, test_iter, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.346 | Test Acc: 90.45%


In [0]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iter, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


Epoch: 01 | Epoch Time: 0m 14s
	Train Loss: 0.110 | Train Acc: 96.22%
	 Val. Loss: 0.363 |  Val. Acc: 90.91%
Epoch: 02 | Epoch Time: 0m 14s
	Train Loss: 0.090 | Train Acc: 96.88%
	 Val. Loss: 0.407 |  Val. Acc: 90.31%
Epoch: 03 | Epoch Time: 0m 14s
	Train Loss: 0.080 | Train Acc: 97.26%
	 Val. Loss: 0.444 |  Val. Acc: 90.50%
Epoch: 04 | Epoch Time: 0m 14s
	Train Loss: 0.066 | Train Acc: 97.75%
	 Val. Loss: 0.479 |  Val. Acc: 90.30%
Epoch: 05 | Epoch Time: 0m 14s
	Train Loss: 0.060 | Train Acc: 97.93%
	 Val. Loss: 0.512 |  Val. Acc: 90.27%
Epoch: 06 | Epoch Time: 0m 14s
	Train Loss: 0.055 | Train Acc: 98.18%
	 Val. Loss: 0.550 |  Val. Acc: 90.23%
Epoch: 07 | Epoch Time: 0m 14s
	Train Loss: 0.051 | Train Acc: 98.31%
	 Val. Loss: 0.573 |  Val. Acc: 89.96%
Epoch: 08 | Epoch Time: 0m 14s
	Train Loss: 0.046 | Train Acc: 98.50%
	 Val. Loss: 0.620 |  Val. Acc: 89.75%
Epoch: 09 | Epoch Time: 0m 14s
	Train Loss: 0.044 | Train Acc: 98.58%
	 Val. Loss: 0.640 |  Val. Acc: 89.80%
Epoch: 10 | Epoch T

In [0]:
# EPOCH = 10

model.load_state_dict(torch.load('tut5-model.pt'))

test_loss, test_acc = evaluate(model, test_iter, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.378 | Test Acc: 90.50%


# RNN

In [0]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = len(LABEL.vocab)

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,876 trainable parameters


In [0]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [0]:
criterion = nn.CrossEntropyLoss() ### changed

In [0]:
model = model.to(device)
criterion = criterion.to(device)

In [0]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iter, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 11s
	Train Loss: 1.385 | Train Acc: 25.31%
	 Val. Loss: 1.386 |  Val. Acc: 24.82%
Epoch: 02 | Epoch Time: 0m 10s
	Train Loss: 1.385 | Train Acc: 24.96%
	 Val. Loss: 1.386 |  Val. Acc: 24.82%
Epoch: 03 | Epoch Time: 0m 10s
	Train Loss: 1.385 | Train Acc: 25.18%
	 Val. Loss: 1.386 |  Val. Acc: 25.19%
Epoch: 04 | Epoch Time: 0m 11s
	Train Loss: 1.385 | Train Acc: 25.23%
	 Val. Loss: 1.386 |  Val. Acc: 25.19%
Epoch: 05 | Epoch Time: 0m 10s
	Train Loss: 1.384 | Train Acc: 25.11%
	 Val. Loss: 1.386 |  Val. Acc: 24.90%


In [0]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iter, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.386 | Test Acc: 25.30%


# LSTM

We'll be using packed padded sequences, which will make our RNN only process the non-padded elements of our sequence, and for any padded element the output will be a zero tensor. To use packed padded sequences, we have to tell the RNN how long the actual sequences are. We do this by setting include_lengths = True for our TEXT field. This will cause batch.text to now be a tuple with the first element being our sentence (a numericalized tensor that has been padded) and the second element being the actual lengths of our sentences.

In [0]:
import random

SEED = 1234 # Random Seed for reproductivity

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField()

In [0]:
from torchtext.data import TabularDataset

# put the train and text csv to PATH
DATA_PATH = '/content/drive/My Drive/lottery-ticket-hypothesis-for-text-classification/data/ag_news_csv/'

train = data.TabularDataset(path = DATA_PATH + 'train.csv', format = 'csv', fields = [('text', TEXT), ('label', LABEL)], skip_header = False)
test = data.TabularDataset(path = DATA_PATH + 'test.csv', format = 'csv', fields = [('text', TEXT), ('label', LABEL)], skip_header = False)

# make splits for data
# by default this splits 70:30
train, valid = train.split(split_ratio = 0.7, random_state = random.seed(SEED))

In [0]:
print(f'Number of training examples: {len(train)}')
print(f'Number of training examples: {len(valid)}')
print(f'Number of testing examples: {len(test)}')

print(vars(train.examples[0]))
print(vars(valid.examples[0]))
print(vars(test.examples[0]))

Number of training examples: 84000
Number of training examples: 36000
Number of testing examples: 7600
{'text': ['In', '30', 'sorties', 'on', 'Monday', ',', 'U.S.', 'helicopters', 'delivered', 'more', 'than', '60,000', 'pounds', 'of', 'water', ',', 'medical', 'supplies', 'and', 'food', '.'], 'label': '1'}
{'text': ['London', '-', 'US', 'sports', 'tycoon', 'Malcolm', 'Glazer', 'was', 'trying', 'to', 'win', 'over', 'Manchester', 'United', '#', '39;s', 'two', 'key', 'Irish', 'shareholders', 'in', 'his', 'bid', 'to', 'buy', 'the', 'English', 'soccer', 'club', ',', 'but', 'had', 'not', 'yet', 'entered', 'serious', 'talks', 'with', 'them', ',', 'sources', 'familiar', 'with', 'the', 'situation', 'said', 'yesterday', '.'], 'label': '2'}
{'text': ['Unions', 'representing', 'workers', 'at', 'Turner', '  ', 'Newall', 'say', 'they', 'are', "'", 'disappointed', "'", 'after', 'talks', 'with', 'stricken', 'parent', 'firm', 'Federal', 'Mogul', '.'], 'label': '3'}


In [0]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train)

print ("Length of Text Vocabulary: " + str(len(TEXT.vocab)))
print ("Vector size of Text Vocabulary: ", TEXT.vocab.vectors.size())
print ("Label Length: " + str(len(LABEL.vocab)))

Length of Text Vocabulary: 25002
Vector size of Text Vocabulary:  torch.Size([25002, 100])
Label Length: 4


In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iter, valid_iter, test_iter = data.BucketIterator.splits(
    (train, valid, test), 
    sort_key = lambda x: x.text,
    batch_size = BATCH_SIZE,
    device = device)

In [0]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths, enforce_sorted=False)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = len(LABEL.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = LSTM(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,812,396 trainable parameters


In [0]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [0]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.4598,  1.1272, -1.1509,  ..., -0.6062,  0.1021,  0.8086],
        [ 0.1967, -2.3082,  0.2116,  ...,  0.9473,  0.1453, -0.7713],
        [ 0.0786,  0.3827, -1.8876,  ..., -0.1305, -0.2890, -0.5978]])

In [0]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.4598,  1.1272, -1.1509,  ..., -0.6062,  0.1021,  0.8086],
        [ 0.1967, -2.3082,  0.2116,  ...,  0.9473,  0.1453, -0.7713],
        [ 0.0786,  0.3827, -1.8876,  ..., -0.1305, -0.2890, -0.5978]])


In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [0]:
criterion = nn.CrossEntropyLoss()
model = model.to(device)
criterion = criterion.to(device)

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()

        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text

            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [0]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iter, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 2m 23s
	Train Loss: 0.528 | Train Acc: 80.80%
	 Val. Loss: 0.339 |  Val. Acc: 88.56%
Epoch: 02 | Epoch Time: 2m 22s
	Train Loss: 0.348 | Train Acc: 87.92%
	 Val. Loss: 0.276 |  Val. Acc: 90.48%
Epoch: 03 | Epoch Time: 2m 23s
	Train Loss: 0.292 | Train Acc: 89.96%
	 Val. Loss: 0.262 |  Val. Acc: 91.19%
Epoch: 04 | Epoch Time: 2m 23s
	Train Loss: 0.260 | Train Acc: 91.02%
	 Val. Loss: 0.250 |  Val. Acc: 91.33%
Epoch: 05 | Epoch Time: 2m 23s
	Train Loss: 0.234 | Train Acc: 91.88%
	 Val. Loss: 0.235 |  Val. Acc: 91.86%


In [0]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iter, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.250 | Test Acc: 91.20%
